# Регуляризация xgboost

###  Более подробно


Данный регуляризатор подобран эвристически и хорошо показывает себя на практике. 

Раскладывая в ряд Тейлора выражение $\mathcal{L}\left(y_i, \hat{y}_i^{\left(t-1\right)} + b_t(x_i)\right)$ до второго порядка, получаем:

$$ Obj^{(t)} = \sum_{i=1}^N\left[\mathcal{L}(y_i, \hat{y}_i^{\left(t-1\right)}) + g_{i}b_{t}(x_i) + \frac{1}{2}h_{i}b_{t}^2(x_i)\right] + \Omega(b_t),$$

где $g_i = \partial_{\hat{y}_i^{(t-1)}} \mathcal{L}(y_i, \hat{y_i}^{(t-1)})$, $h_i = \partial_{\hat{y}_i^{(t-1)}}^2 \mathcal{L}(y_i, \hat{y}_i^{(t-1)}) $ — градиент и гессиан оптимизируемой функции потерь.

Приводя теперь подобные слагаемые и отбрасывая слагаемое $ \mathcal{L}(y_i, \hat{y}_i^{(t-1)}) $, не зависящее от $ b_t(x_i)$ (а следовательно, не влияющее на точку минимума функционала), получаем формулу:
$$ Obj^{(t)} = \Big[\sum_{i=1}^N\mathcal{L}(y_i, \hat{y}_i^{\left(t-1\right)})\Big] + \sum_{i=1}^N\Big[g_{i}b_{t}(x_i) + \frac{1}{2}h_{i}b_{t}^2(x_i)\Big] + \Big[\gamma T + \frac{1}{2}\lambda\sum_{j=1}^{T}w_j^2 + \alpha\sum_{j=1}^{T}|w_j|\Big]$$

Сделаем хитрый трюк: рассмотрим все объекты, через листья, в которые они попали ($I_j$ - множество объектов, попвших в лист $j$)
$$\sum_{i=1}^{N}g_ib_t(x_i) = \sum_{j=1}^{T}w_j\sum_{i\in I_j}g_i = \sum_{j=1}^{T}w_jG_j$$
$$\sum_{i=1}^{N}h_ib_t^2(x_i) = \sum_{j=1}^{T}w^2_j\sum_{i\in I_j}h_i = \sum_{j=1}^{T}w_jH_j$$


Тогда получим

$$ Obj^{(t)} = \Big[\sum_{i=1}^N\mathcal{L}(y_i, \hat{y}_i^{\left(t-1\right)})\Big] + \sum_{j=1}^{T}\left[G_jw_j + \frac{1}{2}(H_j + \lambda)w_j^2 + \alpha|w_j|\right] + \gamma T, $$

Заметим, что первое слагаемое не зависит от текущего оптимизируемого дерева $b_t$, значит его можно отбросить и минимизировать оставшийся функционал

$$ \tilde{Obj}^{(t)} = \sum_{j=1}^{T}\left[G_jw_j + \frac{1}{2}(H_j + \lambda)w_j^2 + \alpha|w_j|\right] + \gamma T \rightarrow \min$$


Теперь, имея заданную структуру дерева, можно аналитически вычислить оптимальные значения для весов.

В общем случае получаем:
$$ w_j = \begin{cases}
-\frac{G_j + \alpha}{H_j + \lambda}, если \space G_j < -\alpha, \\
-\frac{G_j - \alpha}{H_j + \lambda}, если \space G_j > \alpha, \\
0, \space иначе.
\end{cases}
$$

При $\alpha = 0$:
$$ w_j^* = -\frac{G_j}{H_j + \lambda}.$$

Значение функционала при $\alpha = 0$ будет равно:

$$ Obj = -\frac{1}{2}\sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T .$$

Осталось только построить дерево оптимальной структуры. Это можно делать известными методами построения решающих деревьев, проводя разбиения таким образом, чтобы максимизировать gain, определенный как уменьшение $Obj$ в момент этого разбиения. Для уже построенного дерева по формулам $ w_j^* $ вычисляются оптимальные значения в листьях.